In [1]:
#For testing, import the user name
from pro_names import pro_names
from env import api_key
import requests
import pandas as pd
from bs4 import BeautifulSoup
from env import api_key
from time import sleep

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

# Getting New Data

In [3]:
def get_new_patch_data(name_list, api_key):
    """
    This function takes in a list of usernames that are in the same tier and iterates 
    through them to find their puuids.
    It will then use those puuids to grab all of their match ids that are in the newest patch.
    From there, the function will iterate through all of the match ids and grab the timeline
    data for each match as well as the other game data. 
    Finally, this function will incrementally save the returned json information every 10 times.
    This prevents the loss of all data if the function errors out at some point.
    This function returns nothing.
    """
    
    #Set up the basegame number. Any match_ids below this number are part of the old patch
    #and will not be included. We will use the basegame var for comparisons in the loop
    basegame = 4107000001
    
    #Set up the base url for grabbing match_ids
    match_url = 'https://americas.api.riotgames.com/'
    
    #Set up the base url for grabbing puuids
    puuid_base_url = 'https://na1.api.riotgames.com/'
    
    #Set up another counter to keep track of how many users have been completed
    n = 0
    
    #Set up a counter to keep track of how many json files have been completed in total
    num_json = 0
    
    #Set up one more counter to keep track of how many pulls we have made.
    #Can't go over 100 pulls every 2 minutes.
    num_pulls = 0
    
    #Set up empty lists to store timeline and other game data
    timeline_data = []
    game_data = []
    
    #Loop through each name in the given name_list
    #First grab the puuid, then grab the match_ids, then actual match json info
    for name in name_list:
        
        #for testing
        print(f'Beginning Loop for user {name}')
        
        #Update n var
        n += 1
        
        #Leave a status update after every 100 users
        if n % 100 == 0:
            print(f'Completed {n} users of {len(name_list)}!\n')
        
        ########################## Get puuids ##########################
        
        #Set up the query
        query = f'lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}'
        
        #Get the response
        response = requests.get(puuid_base_url + query)
        
        #Update pull count and sleep for 150 seconds if necessary
        num_pulls += 1
        
        if num_pulls % 100 == 0:
            print('\nHit 100 pulls! Sleeping for 150 seconds.')
            sleep(150)
            print('Continuing!\n')
        
        #Check the response code and leave message if something goes wrong
        if response.status_code != 200:
            print(f'Something went wrong! Status Code: {response.status_code}. Did not get puuid for user: {name}')
            print(f'Skipping this user. There are {len(name_list) - n} users left.\n')
            continue
        
        #Grab the user's puuid
        puuid = response.json()['puuid']
        
        #For testing
        print(f'\t-Got the puuid for user {name}.')
        
        ########################## Get Match IDs #########################
        
        #Now that we have a single puuid, grab all the match ids for that user that are greater
        #than the basegame value. These match_ids will be for the newest patch.
        
        #Set up a counter
        i = 0
        
        #This loop will be set up to run indefinitely. Be sure to break out of it when 
        #Done with the current user
        while i >= 0:
            #Set up the query to get a match_id
            query = f'lol/match/v5/matches/by-puuid/{puuid}/ids?start={i}&count=1&api_key={api_key}'
            
            #Get the response
            response = requests.get(match_url + query)
            
            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response status code and leave message if something went wrong
            if response.status_code != 200:
                print(f'Something went wrong! Status Code: {response.status_code}. Did not get Match_ID for user: {name}')
                print(f'Skipping this user. There are {len(name_list) - n} users left.\n')
                break
                
            #Grab the match_id
            match = response.json()
            
            #Verify there's something in it
            if len(match) == 0:
                print(f'\t-No match ID for user {name}. Skipping this user!')
                break
            
            #Convert match_id to integer
            match_num = int(match[0].strip('NA1_'))
            
            #Compare current match_id to basegame var.
            #Skip if match_id is less than basegame
            if match_num < basegame:
                print(f'\t-Completed getting all current patch Match IDs for user {name}')
                print(f'\t-There are {len(name_list) - n} users left.')
                break

            ####################### Get Match Info ##########################
            
            #Now that we know the current match_id is part of the newest patch,
            #We can retrieve the actual match info. Keep in mind this is still part
            #of the while loop above
                
            #Every 500 entries, save the lists as json files and reset the lists
            if len(timeline_data) % 10 == 0:    
                #Save timeline data.
                #First, convert it to a df
                timeline_df = pd.DataFrame(timeline_data)

                #Save as json file. Use username string to identify whose file it is
                timeline_df.to_json(f'timeline_data_user_{name}_end_{num_json}.json')

                #Print status message
                print(f'\t-Created timeline_data_user_{name}_end_{num_json}.json')

                #Save other game data
                #First, convert it to a df
                game_df = pd.DataFrame(game_data)

                #Save as json file. Use username string to identify whose file it is
                game_df.to_json(f'other_game_data_user_{name}_end_{num_json}.json')

                #Print status message
                print(f'\t-Created other_game_data_user_{name}_end_{num_json}.json\n')
                
                #Now reset the temp lists
                timeline_data = []
                game_data = []
            
        
            ############################# TIMELINE DATA #######################
            #Set up timeline url
            timeline_query = f'lol/match/v5/matches/{match[0]}/timeline/?api_key={api_key}'

            #Grab timeline json data
            timeline_response = requests.get(match_url + timeline_query)

            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response and leave a status message
            if timeline_response.status_code != 200:
                print(f'Something went wrong getting TIMELINE DATA! Status Code {timeline_response.status_code} for match ID: {match}.')
                print(f'Skipping this match ID.\n')
                continue

            #Turn it into json format
            timeline_json = timeline_response.json()

            #Append this data to the timeline_data list
            timeline_data.append(timeline_json)
            
            ############################# OTHER GAME DATA #######################
            #Set up game data url
            game_query = f'lol/match/v5/matches/{match[0]}?api_key={api_key}'

            #Grab game json data
            game_response = requests.get(match_url + game_query)

            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response and leave a status message.
            if game_response.status_code != 200:
                print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
                print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')

                #Remove the last entry in the timeline_data list
                timeline_data.pop()
                continue

            #Turn it into json format
            game_json = game_response.json()

            #Append this data to the game_data list
            game_data.append(game_json)
            
            #Update num_json
            num_json += 1
            
            #Update i
            i += 1
            
        #Tell the user how many match_ids were used for the current user
        print(f'\t-Completed {i} Match IDs for user {name}\n')
            
    #Complete! Leave a message
    print('All users completed! Follow suit.')
    
    #Return any json data that was not saved at the end of the function
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)

    #Save as json file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_user_{name}_end_{num_json}.json')

    #Print status message
    print(f'Created timeline_data_user_{name}_end_{num_json}.json')

    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)

    #Save as json file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_user_{name}_end_{num_json}.json')

    #Print status message
    print(f'Created other_game_data_user_{name}_end_{num_json}.json')
    
    #For testing
    return timeline_data, game_data
            

In [9]:
pro_names = [
 'Raes',
 'Isles',
 'Engo',
 'Sarcasm',
 'iconic',
 'Niles',
 'RoseThorn',
 'Yunbee',
 'nxi',
 'haeri',
 'Yeon',
 'Eyla',
 'Thien',
 'Katsurii',
 'Pretty',
 'Yursan',
 'tomio',
 'Spawn',
 'Shoryu',
 'jojopyun',
 'VALUE',
 'Takeover',
 'Hyper',
 'Keel',
 'Gamsu',
 'iNsTiNcT',
 'Danny',
 'Donbray',
 'Philip',
 'TEAMLUKE',
 'Srtty',
 'Bluster',
 'Aspect',
 'SkyTec',
 'H4xDefender',
 'Busio',
 'Chad']

In [10]:
# skipped Rakin

In [11]:
timeline_data, game_data = get_new_patch_data(pro_names, api_key)

Beginning Loop for user Raes
	-Got the puuid for user Raes.
	-Completed getting all current patch Match IDs for user Raes
	-There are 36 users left.
	-Completed 0 Match IDs for user Raes

Beginning Loop for user Isles
	-Got the puuid for user Isles.
	-Created timeline_data_user_Isles_end_0.json
	-Created other_game_data_user_Isles_end_0.json

	-Completed getting all current patch Match IDs for user Isles
	-There are 35 users left.
	-Completed 9 Match IDs for user Isles

Beginning Loop for user Engo
	-Got the puuid for user Engo.
	-Completed getting all current patch Match IDs for user Engo
	-There are 34 users left.
	-Completed 0 Match IDs for user Engo

Beginning Loop for user Sarcasm
	-Got the puuid for user Sarcasm.
	-Created timeline_data_user_Sarcasm_end_10.json
	-Created other_game_data_user_Sarcasm_end_10.json

	-Completed getting all current patch Match IDs for user Sarcasm
	-There are 33 users left.
	-Completed 5 Match IDs for user Sarcasm

Beginning Loop for user iconic
	-Got

In [ ]:
timeline_data = pd.read_json('timeline_data_user_Allorim_end_50.json')
other_game_data_user_Allorim_end_50.json

Convert 

df1 = prepare

timeline_data_user_Aspect_end_60.json
other_game_data_user_Aspect_end_60.json

df2
timeline_data_user_Benji_end_10.json
other_game_data_user_Benji_end_10.json

timeline_data_user_Breezyyy_end_20.json
other_game_data_user_Breezyyy_end_20.json

timeline_data_user_Chad_end_69.json
other_game_data_user_Chad_end_69.json

timeline_data_user_Danny_end_40.json
other_game_data_user_Danny_end_40.json

timeline_data_user_Destiny_end_100.json
other_game_data_user_Destiny_end_100.json

timeline_data_user_Dhokla_end_130.json
other_game_data_user_Dhokla_end_130.json

timeline_data_user_Doublelift_end_60.json
other_game_data_user_Doublelift_end_60.json

timeline_data_user_Dreams_end_70.json
other_game_data_user_Dreams_end_70.json

timeline_data_user_Dyrus_end_0.json
other_game_data_user_Dyrus_end_0.json

timeline_data_user_FakeGod_end_160.json
other_game_data_user_FakeGod_end_160.json

timeline_data_user_Fill_end_140.json
other_game_data_user_Fill_end_140.json

timeline_data_user_Isles_end_0.json
other_game_data_user_Isles_end_0.json

timeline_data_user_Jensen_end_0.json
other_game_data_user_Jensen_end_0.json

timeline_data_user_Jensen_end_10.json
other_game_data_user_Jensen_end_10.json

timeline_data_user_Jiizuke_end_110.json
other_game_data_user_Jiizuke_end_110.json

timeline_data_user_Mash_end_30.json
other_game_data_user_Mash_end_30.json

timeline_data_user_Revenge_end_170.json
other_game_data_user_Revenge_end_170.json

timeline_data_user_RoseThorn_end_20.json
other_game_data_user_RoseThorn_end_20.json

timeline_data_user_Sarcasm_end_10.json
other_game_data_user_Sarcasm_end_10.json

timeline_data_user_Smoothie_end_20.json
other_game_data_user_Smoothie_end_20.json

timeline_data_user_Srtty_end_50.json
other_game_data_user_Srtty_end_50.json

timeline_data_user_Thien_end_30.json
other_game_data_user_Thien_end_30.json

timeline_data_user_Vulcan_end_120.json
other_game_data_user_Vulcan_end_120.json

timeline_data_user_Winston_end_30.json
other_game_data_user_Winston_end_30.json

timeline_data_user_Xerxe_end_80.json
other_game_data_user_Xerxe_end_80.json

timeline_data_user_Xerxe_end_90.json
other_game_data_user_Xerxe_end_90.json

timeline_data_user_Xmithie_end_40.json
other_game_data_user_Xmithie_end_40.json

In [15]:
import json

In [18]:
timeline_data = pd.read_json('timeline_data_user_Ablazeolive_end_150.json')
other_game_data = pd.read_json('other_game_data_user_Ablazeolive_end_150.json')

In [88]:
other_game_data.iloc[0]['info']['gameDuration']

1919

In [99]:
timeline_data = pd.read_json('timeline_data_user_Ablazeolive_end_150.json')
other_game_data = pd.read_json('other_game_data_user_Ablazeolive_end_150.json')

In [100]:
timeline_data = timeline_data.to_dict(orient = 'records')
other_game_data = other_game_data.to_dict(orient = 'records')

In [102]:
len(timeline_data)

10

In [124]:
other_game_data[0]['info']

{'gameCreation': 1637307711000,
 'gameDuration': 1919,
 'gameEndTimestamp': 1637309659177,
 'gameId': 4109130274,
 'gameMode': 'ULTBOOK',
 'gameName': 'teambuilder-match-4109130274',
 'gameStartTimestamp': 1637307739568,
 'gameType': 'MATCHED_GAME',
 'gameVersion': '11.23.409.111',
 'mapId': 11,
 'participants': [{'assists': 12,
   'baronKills': 0,
   'bountyLevel': 0,
   'champExperience': 21656,
   'champLevel': 18,
   'championId': 98,
   'championName': 'Shen',
   'championTransform': 0,
   'consumablesPurchased': 3,
   'damageDealtToBuildings': 2500,
   'damageDealtToObjectives': 5589,
   'damageDealtToTurrets': 2500,
   'damageSelfMitigated': 40743,
   'deaths': 8,
   'detectorWardsPlaced': 3,
   'doubleKills': 0,
   'dragonKills': 0,
   'firstBloodAssist': False,
   'firstBloodKill': False,
   'firstTowerAssist': False,
   'firstTowerKill': False,
   'gameEndedInEarlySurrender': False,
   'gameEndedInSurrender': False,
   'goldEarned': 14380,
   'goldSpent': 14025,
   'individua

In [134]:
df = prepare(timeline_data, other_game_data, 15)

Skipping: 0 due to <20 min or not classic
Skipping: 1 due to <20 min or not classic
Finished with: 2 of 2
Finished with: 3 of 2
Skipping: 4 due to <20 min or not classic
Finished with: 5 of 2
Finished with: 6 of 2
Skipping: 7 due to <20 min or not classic
Skipping: 8 due to <20 min or not classic
Skipping: 9 due to <20 min or not classic


In [135]:
df

,assistsplayer_1,assistsplayer_10,assistsplayer_2,assistsplayer_3,assistsplayer_5,assistsplayer_7,assistsplayer_8,assistsplayer_9,baron_team100,baron_team200,currentGold_1,currentGold_10,currentGold_2,currentGold_3,currentGold_4,currentGold_5,currentGold_6,currentGold_7,currentGold_8,currentGold_9,deathsplayer_1,deathsplayer_10,deathsplayer_2,deathsplayer_3,deathsplayer_4,deathsplayer_5,deathsplayer_6,deathsplayer_7,deathsplayer_8,deathsplayer_9,dragon_team100,dragon_team200,gameDuration,gameEndTimestamp,gameId,gameMode,gameName,gameStartTimestamp,gameType,gameVersion,goldPerSecond_1,goldPerSecond_10,goldPerSecond_2,goldPerSecond_3,goldPerSecond_4,goldPerSecond_5,goldPerSecond_6,goldPerSecond_7,goldPerSecond_8,goldPerSecond_9,inhibs_lost_team200,jungleMinionsKilled_1,jungleMinionsKilled_10,jungleMinionsKilled_2,jungleMinionsKilled_3,jungleMinionsKilled_4,jungleMinionsKilled_5,jungleMinionsKilled_6,jungleMinionsKilled_7,jungleMinionsKilled_8,jungleMinionsKilled_9,killsplayer_10,killsplayer_2,killsplayer_3,killsplayer_4,killsplayer_5,killsplayer_6,killsplayer_7,killsplayer_8,killsplayer_9,level_1,level_10,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,magicDamageDoneToChampions_1,magicDamageDoneToChampions_10,magicDamageDoneToChampions_2,magicDamageDoneToChampions_3,magicDamageDoneToChampions_4,magicDamageDoneToChampions_5,magicDamageDoneToChampions_6,magicDamageDoneToChampions_7,magicDamageDoneToChampions_8,magicDamageDoneToChampions_9,matchId,minionsKilled_1,minionsKilled_10,minionsKilled_2,minionsKilled_3,minionsKilled_4,minionsKilled_5,minionsKilled_6,minionsKilled_7,minionsKilled_8,minionsKilled_9,physicalDamageDoneToChampions_1,physicalDamageDoneToChampions_10,physicalDamageDoneToChampions_2,physicalDamageDoneToChampions_3,physicalDamageDoneToChampions_4,physicalDamageDoneToChampions_5,physicalDamageDoneToChampions_6,physicalDamageDoneToChampions_7,physicalDamageDoneToChampions_8,physicalDamageDoneToChampions_9,queueId,riftherald_team100,timeEnemySpentControlled_1,timeEnemySpentControlled_10,timeEnemySpentControlled_2,timeEnemySpentControlled_3,timeEnemySpentControlled_4,timeEnemySpentControlled_5,timeEnemySpentControlled_6,timeEnemySpentControlled_7,timeEnemySpentControlled_8,timeEnemySpentControlled_9,timestamp,totalDamageDoneToChampions_1,totalDamageDoneToChampions_10,totalDamageDoneToChampions_2,totalDamageDoneToChampions_3,totalDamageDoneToChampions_4,totalDamageDoneToChampions_5,totalDamageDoneToChampions_6,totalDamageDoneToChampions_7,totalDamageDoneToChampions_8,totalDamageDoneToChampions_9,totalGold_1,totalGold_10,totalGold_2,totalGold_3,totalGold_4,totalGold_5,totalGold_6,totalGold_7,totalGold_8,totalGold_9,towers_lost_team100,towers_lost_team200,trueDamageDoneToChampions_1,trueDamageDoneToChampions_10,trueDamageDoneToChampions_2,trueDamageDoneToChampions_3,trueDamageDoneToChampions_4,trueDamageDoneToChampions_5,trueDamageDoneToChampions_6,trueDamageDoneToChampions_7,trueDamageDoneToChampions_8,trueDamageDoneToChampions_9,ward_player_0,ward_player_1,ward_player_10,ward_player_2,ward_player_3,ward_player_4,ward_player_5,ward_player_6,ward_player_7,ward_player_8,ward_player_9,winningTeam,xp_1,xp_10,xp_2,xp_3,xp_4,xp_5,xp_6,xp_7,xp_8,xp_9,assistsplayer_4,assistsplayer_6,inhibs_lost_team100,killsplayer_1,riftherald_team200
0,1.0,2.0,4.0,3.0,2.0,3.0,3.0,3.0,1.0,1.0,1861.0,816.0,945.0,1327.0,235.0,1809.0,1542.0,1218.0,38.0,130.0,1.0,1.0,4.0,2.0,3.0,2.0,1.0,2.0,2.0,1.0,1.0,4.0,2070,1637196199752,4107225350,CLASSIC,teambuilder-match-4107225350,1637194129094,MATCHED_GAME,11.23.409.111,0.0,30.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,96.0,0.0,0.0,0.0,4.0,96.0,0.0,4.0,3.0,1.0,3.0,2.0,1.0,2.0,4.0,1.0,2.0,11.0,8.0,10.0,11.0,8.0,8.0,10.0,10.0,11.0,9.0,97.0,3294.0,185.0,296.0,366.0,1825.0,3845.0,521.0,941.0,52.0,NA1_4107225350,129.0,7.0,18.0,111.0,123.0,14.0,118.0,12.0,143.0,111.0,3904.0,382.0,3893.0,6959.0,6937.0,568.0,129.0,4775.0,3516.0,3704.0,420,2.0,112302.0,43704.0,184140.0,46839.0,50988.0,8093.0,317

In [106]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 181 columns):
 #    Column                            Dtype  
---   ------                            -----  
 0    assistsplayer_1                   float64
 1    assistsplayer_10                  float64
 2    assistsplayer_2                   float64
 3    assistsplayer_3                   float64
 4    assistsplayer_5                   float64
 5    assistsplayer_7                   float64
 6    assistsplayer_8                   float64
 7    assistsplayer_9                   float64
 8    baron_team100                     float64
 9    baron_team200                     float64
 10   currentGold_1                     float64
 11   currentGold_10                    float64
 12   currentGold_2                     float64
 13   currentGold_3                     float64
 14   currentGold_4                     float64
 15   currentGold_5                     float64
 16   currentGold_6               

In [123]:
df.iloc[0].matchId

'NA1_4107225350'

In [174]:
timeline_data = pd.read_json('timeline_data_user_Ablazeolive_end_150.json')
other_game_data = pd.read_json('other_game_data_user_Ablazeolive_end_150.json')

In [175]:
timeline_data = timeline_data.to_dict(orient = 'records')
other_game_data = other_game_data.to_dict(orient = 'records')

In [178]:
time_data = timeline_data[0:4]
other_data = other_game_data[0:4]
time = 15
df = prepare(time_data, other_data, time)

Skipping: 0 due to <20 min or not classic
Skipping: 1 due to <20 min or not classic
Finished with: 2 of 3
Finished with: 3 of 3


In [157]:
df

""


In [177]:
def prepare(timeline_data, other_game_data, time):
    df = pd.DataFrame()
    
    for index, timedata in enumerate(timeline_data):
        
        # Grabbing Game Duration and Game Mode
        game_duration = other_game_data[index]['info']['gameDuration']
        game_mode = str(other_game_data[index]['info']['gameMode'])
                        
        # Checking if game is a classic game and more than 15 min
        if (int(game_duration) >= 1500) & (game_mode == 'CLASSIC'): 
            
            # Creating an empty dictionary
            final_d = {}
            
            # Using timeline data to extract events, up to a specific time, and add them up to create totals
            kda = get_player_kda(timedata, time)
            
            # Saving totals (kda) to final_d
            final_d.update(kda)
            
            # Using timeline data to extract player stats at a specific time
            final_d.update(get_player_stats(timedata, time))
            
            # Using other_game_data to grab additional overall game data
            final_d.update(get_more_info(other_game_data[index], time))
            
            # Save to DataFrame
            df = df.append(final_d, ignore_index=True)
            
            print(f"Finished with: {index} of {len(timeline_data)-1}")
            
        else:
            print(f"Skipping: {index} due to <20 min or not classic")
    return df

In [128]:
def get_more_info(data, time):

    d = {}
    d["gameMode"] = str(data['info']['gameMode'])
    d["gameType"] = str(data['info']['gameType'])
    d['gameVersion'] = str(data['info']['gameVersion'])
    d['gameDuration'] = str(data['info']['gameDuration'])
    d['matchId'] = str(data['metadata']['matchId'])
    d['gameId'] = str(data['info']['gameId'])
    d['gameName'] = str(data['info']['gameName'])
    d['gameStartTimestamp'] = str(data['info']['gameStartTimestamp'])
    d['gameEndTimestamp'] = str(data['info']['gameEndTimestamp'])
    d['queueId'] = str(data['info']['queueId'])
    
    return d

In [111]:
def get_player_kda(data, time):
    
    df = pd.DataFrame()
    for index in range(len(data['info']['frames'])):
        for event in data['info']['frames'][index]['events']:
            if event['type'] == 'CHAMPION_KILL':
                df = df.append(event, ignore_index =True)
            

    df.timestamp = df.timestamp / 60_000

    kills_df = df[(df.type== 'CHAMPION_KILL') & (df.timestamp <= time)]

    d = {}
    mylist = []
    for value in kills_df.assistingParticipantIds:
        if type(value) == list:
            for assist in value:
                mylist.append(assist)
    temp = pd.DataFrame(mylist)

    for index, player in enumerate(kills_df.killerId.value_counts().sort_index()):

        # Grabbing kills and saving values to same dictionary
        d['killsplayer_'+str(int(kills_df.killerId.value_counts().sort_index().index[index]))] =\
        kills_df.killerId.value_counts().sort_index().iloc[index]

        # Grabbing deaths and saving values to same dictionary
    for index, player in enumerate(kills_df.victimId.value_counts().sort_index()):
        d['deathsplayer_'+str(int(kills_df.victimId.value_counts().sort_index().index[index]))] =\
        kills_df.victimId.value_counts().sort_index().iloc[index]

        # Grabbing assists and saving values to same dictionary
    for index, player in enumerate(temp[0].value_counts().sort_index()):
        d['assistsplayer_'+str(temp[0].value_counts().sort_index().index[index])] = temp.value_counts().sort_index().iloc[index]

    df = pd.DataFrame()
    for index in range(len(data['info']['frames'])):
        for event in data['info']['frames'][index]['events']:
            if event['type'] == 'ELITE_MONSTER_KILL':
                df = df.append(event, ignore_index =True)
            elif event['type']== 'GAME_END':
                df = df.append(event, ignore_index =True)

    kills_df = df[df.type == 'ELITE_MONSTER_KILL']
    for index, player in enumerate(kills_df[kills_df.monsterType=='DRAGON'].killerTeamId.value_counts().sort_index()):

        # Grabbing dragons and saving values to same dictionary
        d['dragon_team'+str(int(kills_df[kills_df.monsterType=='DRAGON'].killerTeamId.value_counts().sort_index().index[index]))] =\
        kills_df[kills_df.monsterType=='DRAGON'].killerTeamId.value_counts().sort_index().iloc[index]

    for index, player in enumerate(kills_df[kills_df.monsterType=='RIFTHERALD'].killerTeamId.value_counts().sort_index()):

        # Grabbing riftherald and saving values to same dictionary
        d['riftherald_team'+str(int(kills_df[kills_df.monsterType=='RIFTHERALD'].killerTeamId.value_counts().sort_index().index[index]))] =\
        kills_df[kills_df.monsterType=='RIFTHERALD'].killerTeamId.value_counts().sort_index().iloc[index]

    for index, player in enumerate(kills_df[kills_df.monsterType=='BARON_NASHOR'].killerTeamId.value_counts().sort_index()):

        # Grabbing baron and saving values to same dictionary
        d['baron_team'+str(int(kills_df[kills_df.monsterType=='BARON_NASHOR'].killerTeamId.value_counts().sort_index().index[index]))] =\
        kills_df[kills_df.monsterType=='BARON_NASHOR'].killerTeamId.value_counts().sort_index().iloc[index]

    # Grabbing target, winning team
    d['winningTeam'] = int(df[df.type == 'GAME_END'].winningTeam)

    df = pd.DataFrame()
    for index in range(len(data['info']['frames'])):
        for event in data['info']['frames'][index]['events']:
            if event['type'] == 'WARD_PLACED':
                df = df.append(event, ignore_index =True)

    kills_df = df[df.type == 'WARD_PLACED']
    for index, player in enumerate(kills_df[kills_df.type=='WARD_PLACED'].creatorId.value_counts().sort_index()):

        # Grabbing baron and saving values to same dictionary
        d['ward_player_'+str(int(kills_df[kills_df.type=='WARD_PLACED'].creatorId.value_counts().sort_index().index[index]))] =\
        kills_df[kills_df.type=='WARD_PLACED'].creatorId.value_counts().sort_index().iloc[index]
    df = pd.DataFrame()
    
    for index in range(len(data['info']['frames'])):
        for event in data['info']['frames'][index]['events']:
            if event['type'] == 'BUILDING_KILL':
                df = df.append(event, ignore_index =True)

    kills_df = df[df.type == 'BUILDING_KILL']

    for index, player in enumerate(kills_df[kills_df.buildingType =='TOWER_BUILDING'].teamId.value_counts().sort_index()):

        # Grabbing baron and saving values to same dictionary
        d['towers_lost_team'+str(int(kills_df[kills_df.buildingType =='TOWER_BUILDING'].teamId.value_counts().sort_index().index[index]))] =\
        kills_df[kills_df.buildingType =='TOWER_BUILDING'].teamId.value_counts().sort_index().iloc[index]
    
    for index, player in enumerate(kills_df[kills_df.buildingType =='INHIBITOR_BUILDING'].teamId.value_counts().sort_index()):

        # Grabbing baron and saving values to same dictionary
        d['inhibs_lost_team'+str(int(kills_df[kills_df.buildingType =='INHIBITOR_BUILDING'].teamId.value_counts().sort_index().index[index]))] =\
        kills_df[kills_df.buildingType =='INHIBITOR_BUILDING'].teamId.value_counts().sort_index().iloc[index]


    return d

In [112]:
def get_player_stats(data, time):
    player_stats = []

    #Here, each timeframe represents about one minute
    timeframe = data['info']['frames'][time]
    players = timeframe['participantFrames']
    #Now create a dicitonary to hold the players' stats from this timeframe
    players_dict = {}
    for i in range(1, len(players) + 1):
        #Now create a temp dict to store the current players stats
        #Use formatted strings to automatically increment the player label
        temp_dict = {
            f'currentGold_{i}': players[str(i)]['currentGold'],
            f'magicDamageDoneToChampions_{i}': players[str(i)]['damageStats']['magicDamageDoneToChampions'],
            f'physicalDamageDoneToChampions_{i}': players[str(i)]['damageStats']['physicalDamageDoneToChampions'],
            f'trueDamageDoneToChampions_{i}': players[str(i)]['damageStats']['trueDamageDoneToChampions'],
            f'totalDamageDoneToChampions_{i}': players[str(i)]['damageStats']['totalDamageDoneToChampions'],
            f'goldPerSecond_{i}': players[str(i)]['goldPerSecond'],
            f'jungleMinionsKilled_{i}': players[str(i)]['jungleMinionsKilled'],
            f'level_{i}': players[str(i)]['level'],
            f'minionsKilled_{i}': players[str(i)]['minionsKilled'],
            f'timeEnemySpentControlled_{i}': players[str(i)]['timeEnemySpentControlled'],
            f'totalGold_{i}': players[str(i)]['totalGold'],
            f'xp_{i}': players[str(i)]['xp']
            }
        #Now that I have the current players stats, extend it to the overall players_dict
        players_dict.update(temp_dict)
    #Update the players_dict one more time with the timestamp for the timeframe
    players_dict.update({'timestamp' : timeframe['timestamp']})
    #Append the players_dict to the overall player_stats list of dicts
    player_stats.append(players_dict)
    return player_stats[0]

In [150]:
data = timeline_data[0:3]
time = 15

In [149]:
len(data[0]['info']['frames'])

33

In [152]:
df = pd.DataFrame()
for index in range(len(data[0]['info']['frames'])):
    for event in data[0]['info']['frames'][index]['events']:
        if event['type'] == 'CHAMPION_KILL':
            df = df.append(event, ignore_index =True)


df.timestamp = df.timestamp / 60_000

kills_df = df[(df.type== 'CHAMPION_KILL') & (df.timestamp <= time)]

d = {}
mylist = []
for value in kills_df.assistingParticipantIds:
    if type(value) == list:
        for assist in value:
            mylist.append(assist)
temp = pd.DataFrame(mylist)

for index, player in enumerate(kills_df.killerId.value_counts().sort_index()):

    # Grabbing kills and saving values to same dictionary
    d['killsplayer_'+str(int(kills_df.killerId.value_counts().sort_index().index[index]))] =\
    kills_df.killerId.value_counts().sort_index().iloc[index]

    # Grabbing deaths and saving values to same dictionary
for index, player in enumerate(kills_df.victimId.value_counts().sort_index()):
    d['deathsplayer_'+str(int(kills_df.victimId.value_counts().sort_index().index[index]))] =\
    kills_df.victimId.value_counts().sort_index().iloc[index]

    # Grabbing assists and saving values to same dictionary
for index, player in enumerate(temp[0].value_counts().sort_index()):
    d['assistsplayer_'+str(temp[0].value_counts().sort_index().index[index])] = temp.value_counts().sort_index().iloc[index]

df = pd.DataFrame()
for index in range(len(data['info']['frames'])):
    for event in data['info']['frames'][index]['events']:
        if event['type'] == 'ELITE_MONSTER_KILL':
            df = df.append(event, ignore_index =True)
        elif event['type']== 'GAME_END':
            df = df.append(event, ignore_index =True)

kills_df = df[df.type == 'ELITE_MONSTER_KILL']
for index, player in enumerate(kills_df[kills_df.monsterType=='DRAGON'].killerTeamId.value_counts().sort_index()):

    # Grabbing dragons and saving values to same dictionary
    d['dragon_team'+str(int(kills_df[kills_df.monsterType=='DRAGON'].killerTeamId.value_counts().sort_index().index[index]))] =\
    kills_df[kills_df.monsterType=='DRAGON'].killerTeamId.value_counts().sort_index().iloc[index]

for index, player in enumerate(kills_df[kills_df.monsterType=='RIFTHERALD'].killerTeamId.value_counts().sort_index()):

    # Grabbing riftherald and saving values to same dictionary
    d['riftherald_team'+str(int(kills_df[kills_df.monsterType=='RIFTHERALD'].killerTeamId.value_counts().sort_index().index[index]))] =\
    kills_df[kills_df.monsterType=='RIFTHERALD'].killerTeamId.value_counts().sort_index().iloc[index]

for index, player in enumerate(kills_df[kills_df.monsterType=='BARON_NASHOR'].killerTeamId.value_counts().sort_index()):

    # Grabbing baron and saving values to same dictionary
    d['baron_team'+str(int(kills_df[kills_df.monsterType=='BARON_NASHOR'].killerTeamId.value_counts().sort_index().index[index]))] =\
    kills_df[kills_df.monsterType=='BARON_NASHOR'].killerTeamId.value_counts().sort_index().iloc[index]

# Grabbing target, winning team
d['winningTeam'] = int(df[df.type == 'GAME_END'].winningTeam)

df = pd.DataFrame()
for index in range(len(data['info']['frames'])):
    for event in data['info']['frames'][index]['events']:
        if event['type'] == 'WARD_PLACED':
            df = df.append(event, ignore_index =True)

kills_df = df[df.type == 'WARD_PLACED']
for index, player in enumerate(kills_df[kills_df.type=='WARD_PLACED'].creatorId.value_counts().sort_index()):

    # Grabbing baron and saving values to same dictionary
    d['ward_player_'+str(int(kills_df[kills_df.type=='WARD_PLACED'].creatorId.value_counts().sort_index().index[index]))] =\
    kills_df[kills_df.type=='WARD_PLACED'].creatorId.value_counts().sort_index().iloc[index]
df = pd.DataFrame()

for index in range(len(data['info']['frames'])):
    for event in data['info']['frames'][index]['events']:
        if event['type'] == 'BUILDING_KILL':
            df = df.append(event, ignore_index =True)

kills_df = df[df.type == 'BUILDING_KILL']

for index, player in enumerate(kills_df[kills_df.buildingType =='TOWER_BUILDING'].teamId.value_counts().sort_index()):

    # Grabbing baron and saving values to same dictionary
    d['towers_lost_team'+str(int(kills_df[kills_df.buildingType =='TOWER_BUILDING'].teamId.value_counts().sort_index().index[index]))] =\
    kills_df[kills_df.buildingType =='TOWER_BUILDING'].teamId.value_counts().sort_index().iloc[index]

for index, player in enumerate(kills_df[kills_df.buildingType =='INHIBITOR_BUILDING'].teamId.value_counts().sort_index()):

    # Grabbing baron and saving values to same dictionary
    d['inhibs_lost_team'+str(int(kills_df[kills_df.buildingType =='INHIBITOR_BUILDING'].teamId.value_counts().sort_index().index[index]))] =\
    kills_df[kills_df.buildingType =='INHIBITOR_BUILDING'].teamId.value_counts().sort_index().iloc[index]
    
    


TypeError: list indices must be integers or slices, not str

In [142]:
d

{'killsplayer_1': 1,
 'killsplayer_2': 1,
 'killsplayer_3': 6,
 'killsplayer_4': 1,
 'killsplayer_5': 1,
 'killsplayer_6': 6,
 'killsplayer_7': 3,
 'killsplayer_8': 4,
 'killsplayer_9': 9,
 'killsplayer_10': 1,
 'deathsplayer_1': 4,
 'deathsplayer_2': 5,
 'deathsplayer_3': 3,
 'deathsplayer_4': 6,
 'deathsplayer_5': 5,
 'deathsplayer_7': 1,
 'deathsplayer_8': 4,
 'deathsplayer_9': 3,
 'deathsplayer_10': 2,
 'assistsplayer_1': 2,
 'assistsplayer_2': 2,
 'assistsplayer_3': 2,
 'assistsplayer_4': 3,
 'assistsplayer_5': 3,
 'assistsplayer_6': 1,
 'assistsplayer_7': 6,
 'assistsplayer_8': 4,
 'assistsplayer_9': 1,
 'assistsplayer_10': 10,
 'dragon_team100': 4,
 'dragon_team200': 2,
 'riftherald_team200': 1,
 'baron_team100': 2,
 'winningTeam': 100,
 'ward_player_0': 1,
 'ward_player_1': 107,
 'ward_player_2': 2,
 'ward_player_3': 11,
 'ward_player_4': 11,
 'ward_player_5': 16,
 'ward_player_6': 8,
 'ward_player_7': 10,
 'ward_player_8': 9,
 'ward_player_9': 3,
 'ward_player_10': 29,
 'tower